### Year 3 Topology 4 - Limit Checking - Overloading, Voltage limits

In [1]:
import pandas as pd
import altair as alt

##### Loading

##### Percentage Loading in Branches 

Calculation related can be found in PSSE PAGVI Manual Chapter 6 Power system network simulations - Section 6.6.7 Activity Rate 

%I = MVA(actual)/(MVA(rated)*Vpu)

Vpu -> Bus Voltage at from end 

In [2]:
list_all = []
list_gens = [0,50,100]
list_lsc = ['LLS','RLS','HLS']
list_gen_hydro = [400,500,600] #dry, average, wet hydological scenarios
for gen in list_gens:
    for gen_hy in list_gen_hydro:
        for lsc in list_lsc:
            file_csv_out = 'savnw_sol_' + str(gen) +'_hy_' +str(gen_hy) +'_' + str(lsc)+'_l.csv'
            data_load = pd.read_csv(file_csv_out,header=1, skiprows=0)
            # Extracted data contains branch loading from the metered and non-metered ends. The data is filtered to obtain the branch loading from the metered end.
            data_fil = data_load
            data_fil['From Bus V'] = data_fil['From Bus Ext'].str.extract(r'(\d+\.\d+)') 
            data_fil['To Bus V'] = data_fil['To Bus Ext'].str.extract(r'(\d+\.\d+)')
            data_fil['Scenario']= 'Solar = ' + str(gen) + ' MW, ' +'Hydro = ' +str(gen_hy) +' MW, ' + lsc.upper()
            list_all.append(data_fil)
data_all = pd.concat(list_all).reset_index(drop=True)
data_all = data_all[['Scenario','From Bus', 'From Bus Ext', 'To Bus', 'To Bus Ext', 'Ckt ID',
       'Non Metered Bus', ' Branch I', 'Branch Flow MW', 'Branch Flow MVAR',
       'Branch MVA', 'MW Loss', 'MVAR Loss', 'Rate', '%I', '%MVA',
       'From Bus V', 'To Bus V']]

##### Line and Transformer Loading

Transformer and branch loading are categorised to following brackets based on the range to which it falls into 
* Loading less than 25%
* Loading 25% to 50%
* Loading 50% to 70%
* Loading 70% to 80%
* Loading 80% to 85%
* Loading 85% to 90%
* Loading 90% to 95%
* Loading 95% to 100%
* Loading 100% and above

Separating transformer and line branch loadings

In [3]:
transformer_loading = data_all[data_all['From Bus V']!=data_all['To Bus V']].reset_index(drop=True)
branch_loading = data_all[data_all['From Bus V'] == data_all['To Bus V']].reset_index(drop=True)

Defining loading brackets for transformer and line branches 

In [4]:
data = transformer_loading[['Scenario','From Bus', 'From Bus Ext', 'To Bus', 'To Bus Ext', 'Ckt ID','Branch Flow MW', 'Branch Flow MVAR','Branch MVA', 'Rate','%MVA']]
data['Loading'] = ''
data.loc[(data['%MVA'] >100), 'Loading'] = '%MVA>100'
data.loc[(data['%MVA'] <100), 'Loading'] = '95<%MVA<100'
data.loc[(data['%MVA'] <95), 'Loading'] = '90<%MVA<95'
data.loc[(data['%MVA'] <90), 'Loading'] = '85<%MVA<90'
data.loc[(data['%MVA'] <85), 'Loading'] = '80<%MVA<85'
data.loc[(data['%MVA'] <80), 'Loading'] = '70<%MVA<80'
data.loc[(data['%MVA'] <70), 'Loading'] = '50<%MVA<70'
data.loc[(data['%MVA'] <50), 'Loading'] = '25<%MVA<50'
data.loc[(data['%MVA'] <25), 'Loading'] = '%MVA<25'

<ipython-input-4-7f16dd71236d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Loading'] = ''


In [5]:
data_l = branch_loading[['Scenario','From Bus', 'From Bus Ext', 'To Bus', 'To Bus Ext', 'Ckt ID','Branch Flow MW', 'Branch MVA', 'Rate','%I']]
data_l['Loading'] = ''
data_l.loc[(data_l['%I'] >100), 'Loading'] = '%I>100'
data_l.loc[(data_l['%I'] <100), 'Loading'] = '95<%I<100'
data_l.loc[(data_l['%I'] <95), 'Loading'] = '90<%I<95'
data_l.loc[(data_l['%I'] <90), 'Loading'] = '85<%I<90'
data_l.loc[(data_l['%I'] <85), 'Loading'] = '80<%I<85'
data_l.loc[(data_l['%I'] <80), 'Loading'] = '70<%I<80'
data_l.loc[(data_l['%I'] <70), 'Loading'] = '50<%I<70'
data_l.loc[(data_l['%I'] <50), 'Loading'] = '25<%I<50'
data_l.loc[(data_l['%I'] <25), 'Loading'] = '%I<25'

<ipython-input-5-abcf08d21c83>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_l['Loading'] = ''


##### Histogram Transformer Branch Loading 

A histogram for each scenario is plotted to check the no of transformer branches in the above defined loading bracket 

In [6]:
alt.Chart(data).mark_bar().encode(
    alt.Y('Loading',scale = alt.Scale(domain=['%MVA>100','95<%MVA<100','90<%MVA<95','85<%MVA<90','80<%MVA<85','70<%MVA<80','50<%MVA<70','25<%MVA<50','%MVA<25'])),
    alt.X('count()')).properties(height = 120, width = 250).facet('Scenario',columns=5).resolve_scale(x='independent', y = 'independent')

alt.FacetChart(...)

It can be seen that for most of scenarios the transformer branch loading falls into a bracket of loading between 25 to 50 percentage. 

##### Histogram Line Branch Loading 

A histogram is plotted to check the no of line branches in the above defined loading bracket 

In [7]:
alt.Chart(data_l).mark_bar().encode(
    alt.Y('Loading', scale =alt.Scale(domain=['%I>100','95<%I<100','90<%I<95','85<%I<90','80<%I<85','70<%I<80','50<%I<70','25<%I<50','%I<25'])),
    alt.X('count()')).properties(height = 120, width = 250).facet('Scenario',columns=5).resolve_scale(x='independent', y = 'independent')

alt.FacetChart(...)

It can be seen that for most of scenarios the line branch loading falls into a bracket of loading between 25% to 50%

In the following section the lines and transformer branches with loading greater than 80% is filtered 

##### Checking transformer loading greater than 80% 

In [8]:
transformer_overload  = transformer_loading[transformer_loading['%MVA'] > 80.0].reset_index(drop=True)
transformer_overload['Branch'] = transformer_overload['From Bus'].astype(str) + ' ' + transformer_overload['From Bus Ext'] + ' to ' + \
transformer_overload['To Bus'].astype(str) + ' '+ transformer_overload['To Bus Ext'] + ' ' + transformer_overload['Ckt ID'].astype(str)
list_branch = transformer_overload['Branch'].unique()
print(list_branch)

['3018 CATDOG_G    13.800 to 3008 CATDOG      230.00 1']


In [9]:
transformer_overload

,Scenario,From Bus,From Bus Ext,To Bus,To Bus Ext,Ckt ID,Non Metered Bus,Branch I,Branch Flow MW,Branch Flow MVAR,Branch MVA,MW Loss,MVAR Loss,Rate,%I,%MVA,From Bus V,To Bus V,Branch
0,"Solar = 0 MW, Hydro = 400 MW, LLS",3018,CATDOG_G 13.800,3008,CATDOG 230.00,1,3018,4779,90.0,80.0,120.42,0.03,11.09,150.0,76.16,80.28,13.800,230.00,3018 CATDOG_G 13.800 to 3008 CATDOG 23...
1,"Solar = 0 MW, Hydro = 400 MW, RLS",3018,CATDOG_G 13.800,3008,CATDOG 230.00,1,3018,4793,90.0,80.0,120.42,0.03,11.16,150.0,76.39,80.28,13.800,230.00,3018 CATDOG_G 13.800 to 3008 CATDOG 23...
2,"Solar = 0 MW, Hydro = 400 MW, HLS",3018,CATDOG_G 13.800,3008,CATDOG 230.00,1,3018,4806,90.0,80.0,120.42,0.03,11.22,150.0,76.58,80.28,13.800,230.00,3018 CATDOG_G 13.800 to 3008 CATDOG 23...
3,"Solar = 0 MW, Hydro = 500 MW, LLS",3018,CATDOG_G 13.800,3008,CATDOG 230.00,1,3018,4783,90.0,80.0,120.42,0.03,11.11,150.0,76.22,80.28,13.800,230.00,3018 CATDOG_G 13.800 to 3008 CATDOG 23...
4,"Solar = 0 MW, Hydro = 500 MW, RLS",3018,CATDOG_G 13.800,3008,CATDOG 230.00,1,3018,4798,90.0,80.0,120.42,0.03,11.18,150.0,76.46,80.28,13.800,230.00,3018 CATDOG_G 13.800 to 3008 CATDOG 23...
5,"Solar = 0 MW, Hydro = 500 MW, HLS",3018,CATDOG_G 13.800,3008,CATDOG 230.00,1,3018,4819,90.0,80.0,120.41,0.03,11.28,150.0,76.80,80.28,13.800,230.00,3018 CATDOG_G 13.800 to 3008 CATDOG 23...
6,"Solar = 0 MW, Hydro = 600 MW, LLS",3018,CATDOG_G 13.800,3008,CATDOG 230.00,1,3018,4788,90.0,80.0,120.42,0.03,11.13,150.0,76.30,80.28,13.800,230.00,3018 CATDOG_G 13.800 to 3008 CATDOG 23...
7,"Solar = 0 MW, Hydro = 600 MW, RLS",3018,CATDOG_G 13.800,3008,CATDOG 230.00,1,3018,4803,90.0,80.0,120.42,0.03,11.20,150.0,76.54,80.28,13.800,230.00,3018 CATDOG_G 13.800 to 3008 CATDOG 23...
8,"Solar = 0 MW, Hydro = 600 MW, HLS",3018,CATDOG_G 13.800,3008,CATDOG 230.00,1,3018,4863,90.0,80.0,120.42,0.03,11.48,150.0,77.49,80.28,13.800,230.00,3018 CATDOG_G 13.800 to 3008 CATDOG 23...
9,"Solar = 50 MW, Hydro = 400 MW, LLS",3018,CATDOG_G 13.800,3008,CATDOG 230.00,1,3018,4777,90.0,80.0,120.42,0.03,11.08,150.0,76.13,80.28,13.800,230.00,3018 CATDOG_G 13.800 to 3008 CATDOG 23...


##### Transformer branch overload plot

In [10]:
alt.Chart(transformer_overload).mark_line().encode(
    alt.X('%MVA', axis=alt.Axis(format='.3f'),scale=alt.Scale(zero=False)),
    alt.Y('Scenario'),
    alt.Color('Branch', legend=None)).properties(height = 300, width =400).facet('Branch',columns=2).resolve_scale(x='independent', y = 'independent')

alt.FacetChart(...)

##### Checking branch loading greater than 80% 

In [11]:

branch_overload = branch_loading[branch_loading['%I'] > 80.0].reset_index(drop=True)
branch_overload['Branch'] = branch_overload['From Bus'].astype(str) + ' ' + branch_overload['From Bus Ext'] + ' to ' + \
branch_overload['To Bus'].astype(str) + ' '+ branch_overload['To Bus Ext'] + ' ' + branch_overload['Ckt ID'].astype(str)
list_branch = branch_overload['Branch'].unique()
print(list_branch)


['154 DOWNTN      230.00 to 203 EAST230     230.00 1'
 '203 EAST230     230.00 to 154 DOWNTN      230.00 1']


In [12]:
branch_overload

,Scenario,From Bus,From Bus Ext,To Bus,To Bus Ext,Ckt ID,Non Metered Bus,Branch I,Branch Flow MW,Branch Flow MVAR,Branch MVA,MW Loss,MVAR Loss,Rate,%I,%MVA,From Bus V,To Bus V,Branch
0,"Solar = 0 MW, Hydro = 400 MW, RLS",154,DOWNTN 230.00,203,EAST230 230.00,1,203,403,-148.20,-49.84,156.36,1.01,10.15,200.0,80.37,78.18,230.00,230.00,154 DOWNTN 230.00 to 203 EAST230 230....
1,"Solar = 0 MW, Hydro = 400 MW, HLS",154,DOWNTN 230.00,203,EAST230 230.00,1,203,410,-151.11,-48.88,158.82,1.05,10.52,200.0,81.78,79.41,230.00,230.00,154 DOWNTN 230.00 to 203 EAST230 230....
2,"Solar = 0 MW, Hydro = 400 MW, HLS",203,EAST230 230.00,154,DOWNTN 230.00,1,203,402,152.16,49.71,160.08,1.05,10.52,200.0,80.26,80.04,230.00,230.00,203 EAST230 230.00 to 154 DOWNTN 230....
3,"Solar = 0 MW, Hydro = 500 MW, LLS",154,DOWNTN 230.00,203,EAST230 230.00,1,203,430,-160.66,-45.89,167.08,1.16,11.59,200.0,85.76,83.54,230.00,230.00,154 DOWNTN 230.00 to 203 EAST230 230....
4,"Solar = 0 MW, Hydro = 500 MW, LLS",203,EAST230 230.00,154,DOWNTN 230.00,1,203,423,161.82,47.74,168.71,1.16,11.59,200.0,84.38,84.36,230.00,230.00,203 EAST230 230.00 to 154 DOWNTN 230....
5,"Solar = 0 MW, Hydro = 500 MW, RLS",154,DOWNTN 230.00,203,EAST230 230.00,1,203,439,-164.20,-44.58,170.14,1.21,12.08,200.0,87.51,85.07,230.00,230.00,154 DOWNTN 230.00 to 203 EAST230 230....
6,"Solar = 0 MW, Hydro = 500 MW, RLS",203,EAST230 230.00,154,DOWNTN 230.00,1,203,432,165.40,46.96,171.94,1.21,12.08,200.0,86.19,85.97,230.00,230.00,203 EAST230 230.00 to 154 DOWNTN 230....
7,"Solar = 0 MW, Hydro = 500 MW, HLS",154,DOWNTN 230.00,203,EAST230 230.00,1,203,448,-166.97,-44.90,172.90,1.26,12.60,200.0,89.33,86.45,230.00,230.00,154 DOWNTN 230.00 to 203 EAST230 230....
8,"Solar = 0 MW, Hydro = 500 MW, HLS",203,EAST230 230.00,154,DOWNTN 230.00,1,203,441,168.23,47.88,174.91,1.26,12.60,200.0,88.02,87.46,230.00,230.00,203 EAST230 230.00 to 154 DOWNTN 230....
9,"Solar = 0 MW, Hydro = 600 MW, LLS",154,DOWNTN 230.00,203,EAST230 230.00,1,203,467,-176.61,-40.31,181.15,1.37,13.70,200.0,93.04,90.58,230.00,230.00,154 DOWNTN 230.00 to 203 EAST230 230....


Transmission line branch overload plot

In [13]:
alt.Chart(branch_overload).mark_line().encode(
    alt.X('%I', scale=alt.Scale(zero=False)),
    alt.Y('Scenario', sort='-x'),
    alt.Color('Branch', legend=None)).properties(height = 300, width =400).facet('Branch',columns=3).resolve_scale(x='independent', y = 'independent')

alt.FacetChart(...)

##### Bus Voltage

In [14]:
list_volt = []
list_gens = [0,50,100]
list_lsc = ['LLS','RLS','HLS']
list_gen_hydro = [400,500,600] #dry, average, wet hydological scenarios
for gen in list_gens:
    for gen_hy in list_gen_hydro:
        for lsc in list_lsc:
            file_sav_in = 'savnw_sol_' + str(gen) +'_hy_' +str(gen_hy) +'_' + str(lsc)+'_volt.csv'
            data_volt = pd.read_csv(file_sav_in,header=1, skiprows=0)
            data_volt['Scenario']= 'Solar = ' + str(gen) + ' MW, ' +'Hydro = ' +str(gen_hy) +' MW, ' + lsc.upper()
            list_volt.append(data_volt)
data_all_volt = pd.concat(list_volt).reset_index(drop=True)
data_all_volt = data_all_volt[['Scenario','Bus Number', 'Bus Name', 'Area', 'Base Voltage', 'Bus Voltage(PU)','Bus Voltage(kV)']]

Similar to branch loading categorization into brackets of loadings, voltages are also categorised into brackets

In [15]:
data_v = data_all_volt
data_v['Voltage Range'] = ''
data_v.loc[(data_v['Bus Voltage(PU)'] >1.1), 'Voltage Range'] = 'Vbus>1.1'
data_v.loc[(data_v['Bus Voltage(PU)'] <1.1), 'Voltage Range'] = '1.05<Vbus<1.1'
data_v.loc[(data_v['Bus Voltage(PU)'] <1.05), 'Voltage Range'] = '1.0<Vbus<1.05'
data_v.loc[(data_v['Bus Voltage(PU)'] <1.0), 'Voltage Range'] = '0.95<Vbus<1.0'
data_v.loc[(data_v['Bus Voltage(PU)'] <0.95), 'Voltage Range'] = '0.95<Vbus<0.9'
data_v.loc[(data_v['Bus Voltage(PU)'] <0.9), 'Voltage Range'] = 'Vbus<0.9'

##### Histogram Bus Voltages

A histogram is plotted to check the no of Buses in each of the above defined brackets 

In [16]:
alt.Chart(data_v).mark_bar().encode(
    alt.Y('Voltage Range', scale =alt.Scale(domain=['Vbus>1.1','1.05<Vbus<1.1','1.0<Vbus<1.05','0.95<Vbus<1.0','0.95<Vbus<0.9','Vbus<0.9'])),
    alt.X('count()')).properties(height = 120, width = 250).facet('Scenario',columns=5).resolve_scale(x='independent', y = 'independent')

alt.FacetChart(...)

It can be seen that most of the buses have voltages in the bracket of 1.0 PU and 1.05 PU

In the following section, the buses out of upper and lower normal operating range is filtered and reported 

##### Voltage Upper Limits 

In [17]:
data_all_volt[data_all_volt['Bus Voltage(PU)']>1.05].reset_index(drop=True)['Bus Number'].unique()

array([3018,  206,  211,  212], dtype=int64)

In [18]:
volt_high = data_all_volt[data_all_volt['Bus Voltage(PU)']>1.05].reset_index(drop=True)
volt_high

,Scenario,Bus Number,Bus Name,Area,Base Voltage,Bus Voltage(PU),Bus Voltage(kV),Voltage Range
0,"Solar = 0 MW, Hydro = 400 MW, LLS",3018,CATDOG_G,5,13.8,1.054,14.547,1.05<Vbus<1.1
1,"Solar = 0 MW, Hydro = 400 MW, RLS",3018,CATDOG_G,5,13.8,1.051,14.502,1.05<Vbus<1.1
2,"Solar = 0 MW, Hydro = 500 MW, LLS",3018,CATDOG_G,5,13.8,1.053,14.534,1.05<Vbus<1.1
3,"Solar = 0 MW, Hydro = 500 MW, HLS",206,URBGEN,2,18.0,1.053,18.946,1.05<Vbus<1.1
4,"Solar = 0 MW, Hydro = 600 MW, LLS",3018,CATDOG_G,5,13.8,1.052,14.520,1.05<Vbus<1.1
5,"Solar = 0 MW, Hydro = 600 MW, RLS",206,URBGEN,2,18.0,1.053,18.949,1.05<Vbus<1.1
6,"Solar = 0 MW, Hydro = 600 MW, HLS",211,HYDRO_G,2,20.0,1.052,21.046,1.05<Vbus<1.1
7,"Solar = 0 MW, Hydro = 600 MW, HLS",212,HYDRO_N,2,20.0,1.052,21.046,1.05<Vbus<1.1
8,"Solar = 50 MW, Hydro = 400 MW, LLS",3018,CATDOG_G,5,13.8,1.055,14.552,1.05<Vbus<1.1
9,"Solar = 50 MW, Hydro = 400 MW, RLS",3018,CATDOG_G,5,13.8,1.051,14.508,1.05<Vbus<1.1


##### Upper voltage limit violation plot

In [19]:
alt.Chart(volt_high).mark_line().encode(
    alt.X('Bus Voltage(PU)', axis=alt.Axis(format='.3f'), scale=alt.Scale(nice=False, zero=False)),
    alt.Y('Scenario', sort='-x'),
    alt.Color('Bus Number:N', legend=None)).properties(height = 150, width = 250).facet('Bus Number:N',columns=2).resolve_scale(x='independent', y = 'independent')

alt.FacetChart(...)

##### Voltage Lower Limits

In [20]:
data_all_volt[data_all_volt['Bus Voltage(PU)']<0.95]['Bus Number'].unique()

array([], dtype=int64)

In [21]:
data_all_volt[data_all_volt['Bus Voltage(PU)']<0.95]

,Scenario,Bus Number,Bus Name,Area,Base Voltage,Bus Voltage(PU),Bus Voltage(kV),Voltage Range


#### Conclusion 

Year 3 Toplogy 4 limit violation check for overloading and voltage was carried out for the three load scenarios. It was seen that
1. Overloading of greater than 80% was reported for the branch ' 154 DOWNTN      230.00 to 203 EAST230     230.00 1'. and no transformers were overloaded for considered load scenarios. 
   For the branch ' 154 DOWNTN      230.00 to 203 EAST230     230.00 1' the scenarios of overload > 80% are
    * Solar = 0 MW, Hydro = 400 MW, RLS, HLS
    * Solar = 0 MW, Hydro = 500 MW, LLS, RLS, HLS
    * Solar = 0 MW, Hydro = 600 MW, LLS, RLS, HLS
    * Solar = 50 MW, Hydro = 500 MW, LLS, RLS, HLS
    * Solar = 50 MW, Hydro = 600 MW, LLS, RLS, HLS
    * Solar = 100 MW, Hydro = 500 MW, LLS, RLS, HLS
    * Solar = 100 MW, Hydro = 600 MW, LLS, RLS, HLS
2. The buses violating the upper voltage limit for various scenarios are:
    * Solar = 0 MW, Hydro = 500 MW, HLS	
      206	URBGEN
    * Solar = 0 MW, Hydro = 600 MW, HLS	
      211	HYDRO_G, 212	HYDRO_N	
    * Solar = 50 MW, Hydro = 500 MW, HLS	
      206	URBGEN
    * Solar = 100 MW, Hydro = 500 MW, HLS
      206	URBGEN